# Estimator
- 모델 구현에만 집중할 수 있는 환경을 제공하는 Tensorflow 고수준의 API
- 기본 제공 기능
  - 학습(Train) : 정의한 모델 파라미터에 대해 학습
  - 평가(Evaluate) : 학습한 모델에 대한 성능을 측정
  - 예측(Predict) : 모델을 통해 입력값에 대한 예측값을 받는다.
  - 배포(Export) : 사용할 모델을 바이너리 파일로 출력한다.
- 기본 제공 모델
  - 선형 회귀, 선형 분류, 심층 신경망 분류기, 심층 신경망 회귀 모델 등



## 모델 함수 구현 및 학습, 검증, 예측
- 모델 함수의 인자는 총 5개이다.
  - features : 모델에 적용되는 입력 값. 학습, 검증, 예측 과정 모두에 사용
  - labels : 모델의 정답 라벨값을 의미한다.
  - mode : 현재 모델 함수가 실행된 모드(학습, 검증, 예측)를 의미한다.
  - params : 모델에 적용될 부가적인 하이퍼파라미터 값을 의미한다.
  - config : 모델에 적용할 설정값을 의미한다.



In [0]:
# 모델 함수 구현 예시
def model_fn(featrues, labels, mode, params, config):
  
  # 모델 구현 부분
  
  return tf. estimator.EstimatorSpec(...)

tf. estimator.Estimator(model_fn=model_fn, model_dir=...,configs=...,params=...)

# 모델 학습 
estimator.train()

# 학습한 모델 검증
estimator.evaluate(input_fn=...)

# 학습한 모델을 통한 예측
estimator.predict(input_fn=...)

def train_input_fn():
  
  # 데이터 파이프라인 구현 부분 # 데이터의 셔플, 배치, 반복 등의 기능이 들어간다.
  
  return features, labels

## 에스티메이터를 활용한 심층 신경망 분류기 사용 구현
- 텍스트의 긍/부정을 예측하는 감정 분석(Sentiment Anlaysis) 모델 구현

In [0]:
# 전처리
import tensorflow as tf
from tensorflow.keras import preprocessing

samples=['너 오늘 이뻐 보인다',
        '나는 오늘 기분이 더러워',
        '끝내주는데, 좋은 일이 있나봐',
        '나 좋은 일이 생겼어',
        '아 오늘 진짜 짜증나',
        '환상적인데, 정말 좋은거 같아']
labels = [[1],[0],[1],[1],[0],[1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [0]:
# 데이터 입력 함수 정의
EPOCH = 100

def train_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
  dataset = dataset.repeat(EPOCH)
  dataset = dataset.batch(1)
  dataset = dataset.shuffle(len(sequences))
  iterator = dataset.make_one_shot_iterator()
  
  return iterator.get_next()

In [0]:
# 모델 함수 구현
VOCAB_SIZE = len(word_index)+1 # 전체 단어 개수
EMB_SIZE = 128 # 임베딩 벡터 사이즈 
# Word Embedding은 Word를 R차원의 Vector로 매핑시켜주는 것을 말한다.

def model_fn(features,labels,mode):
  TRAIN = mode = tf.estimator.ModeKeys.TRAIN
  EVAL = mode = tf.estimator.ModeKeys.EVAL
  PREDICT = mode = tf.estimator.ModeKeys.PREDICT
  
  embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
  embed_input = tf.reduce_mean(embed_input, axis=1)
  
  hidden_layer = tf.keras.layers.Dense(128,activation=tf.nn.relu)(embed_input)
  output_layer = tf.keras.layers.Dense(1)(hidden_layer)
  output = tf.nn.sigmoid(output_layer)
  
  loss = tf.losses.mean_squared_error(output,labels) # 손실 함수 : MSE 사용
  
  if TRAIN:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss,global_step) # optimizer : Adam 사용
    return tf.estimator.EstimatorSpec(mode = mode, train_op=train_op, loss=loss)

In [21]:
# 학습
DATA_OUT_PATH = './data_out/'

import os

if not os.path.exists(DATA_OUT_PATH):
  os.makedirs(DATA_OUT_PATH)

estimator = tf.estimator.Estimator(model_fn=model_fn, model_dir=DATA_OUT_PATH + 'checkpoint/dnn')
estimator.train(train_input_fn)
estimator.predict(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffaf067e710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

<generator object Estimator.predict at 0x7ffaf0d67938>